In [ ]:
# Plot zones where these things are true vs. radius (just stripes on an x-axis or something):
# instead of using a fixed number for the inequality, use the m2 quantity
# (i) t_conv < t_inspiral & L_conv < L_drag
# (ii) t_conv < t_inspiral & L_conv > L_drag
# (iii) t_conv > t_inspiral

In [1]:
# import as needed
import mesa_reader as mr
import matplotlib.pylab as plt
import numpy as np
import os
from math import log
from scipy.integrate import cumtrapz
import math

from ipynb.fs.full.profileFunctions import getMaxRadiusProfile
from ipynb.fs.full.tConvFunctions import getTConv
from ipynb.fs.full.maxLumFunctions import getMaxLum
from ipynb.fs.full.functions import getTInspiral 
from ipynb.fs.full.dragLumFunctions import getDragLum
from ipynb.fs.full.miscFunctions import getR2

G = 6.67408e-11 # gravitational constant
# change G to cgs units
G = G * 1e3

In [3]:
# go ahead and import radius for everything
def getRadius(p, localSim):
    from math import log
    if localSim:
            r = p.logR       # bring in the radius
            r = 10**r        # un-log it

    else:
        r = p.radius     # bring in the radius

    r = r*69.551e9   # change units to cm
    r = r[100:]      # cut off the first 100 points - takes care of surface weirdness
    
    return r

In [4]:
# determine which situation it is
def findZone(tconv, lconv, tinsp, ldrag):
    if ((tconv < tinsp) and (lconv < ldrag)):
        return 'removeAll'
    
    elif ((tconv < tinsp) and (lconv > ldrag)):
        return 'removeLimited'
    
    elif (tconv > tinsp):
        return 'removeNone'

# just get the maximum radius
def getMaxRadius(rList):
    return rList[0]

# normalize the radius by dividing by the max radius
def normalizeRadius(r, maxR):
    normR = r / maxR
    return normR

In [ ]:
# keep m2 constant and compare multiple m1 values
def plotConvZonesConstSec(p, localSim, m1, m2, ypos):
    """Plots convective zones for multiple primary masses given constant secondary mass.
    
    Keyword arguments:
    p -- MESA profile
    localSim -- boolean; if the profile was generated locally
    m1 -- mass of the primary
    m2 -- mass of the secondary
    ypos -- where on the y-axis the data is plotted
    """
    import mesa_reader as mr
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from math import log
    from scipy.integrate import cumtrapz
    import math

    from ipynb.fs.full.profileFunctions import getMaxRadiusProfile
    from ipynb.fs.full.tConvFunctions import getTConv
    from ipynb.fs.full.maxLumFunctions import getMaxLum
    from ipynb.fs.full.functions import getTInspiral 
    from ipynb.fs.full.dragLumFunctions import getDragLum
    from ipynb.fs.full.miscFunctions import getR2
    
    # internal
    from ipynb.fs.full.convZoneFunctions import getRadius
    from ipynb.fs.full.convZoneFunctions import findZone
    from ipynb.fs.full.convZoneFunctions import getMaxRadius
    from ipynb.fs.full.convZoneFunctions import normalizeRadius
    
    r = getRadius(p, localSim)
    tconv = getTConv(p)
    maxLum = getMaxLum(p, localSim)
    tinsp = getTInspiral(p, m2)
    dragLum = getDragLum(p, m2)
    
    # tconv is always awkwardly short
    r = r[:-2]

    maxR = getMaxRadius(r)
    n = normalizeRadius(r, maxR)
    
    coreMass = p.he_core_mass + p.c_core_mass + p.o_core_mass + p.si_core_mass + p.fe_core_mass
    coreMass = coreMass*1.989e33
    
    # cut at rshred
    r2 = getR2(m2)
    m2 = m2*1.989e33 # units
    r2 = r2*69.551e9 # units
    rshred = r2 * (2*coreMass/m2)**(1/3)
    
    i = 0
    for x in r:
        if x > rshred:
            i+=1
    
    maxLum = np.flip(maxLum)
    tinsp = np.flip(tinsp)
    
    r = r[:i]
    n = n[:i]
    tconv = tconv[:i]
    maxLum = maxLum[:i]
    tinsp = tinsp[:i]
    dragLum = dragLum[:i]

    for i in range(len(r)):
        s = findZone(tconv[i], maxLum[i], tinsp[i], dragLum[i])

        color = ''
        name = ''

        if (s == 'removeAll'):
            color = 'g+'

        elif (s == 'removeLimited'):
            color = 'b+'

        elif (s == 'removeNone'):
            color = 'r+'
            
        plt.plot(n[i], ypos, color)

In [ ]:
# keep m1 constant and compare multiple m2 values

def plotConvZonesConstPri(p, localSim, m1, m2, ypos):
    """Plots convective zones for multiple secondary masses given constant primary mass.
    
    Keyword arguments:
    p -- the m1 MESA profile being used
    localSim -- boolean; local or web simulation
    m1 -- mass of the primary star (should be removed soon)
    m2 -- mass of the secondary star
    ypos -- where on the y-axis the data is plotted
    """
    import mesa_reader as mr
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from math import log
    from scipy.integrate import cumtrapz
    import math

    from ipynb.fs.full.profileFunctions import getMaxRadiusProfile
    from ipynb.fs.full.tConvFunctions import getTConv
    from ipynb.fs.full.maxLumFunctions import getMaxLum
    from ipynb.fs.full.functions import getTInspiral 
    from ipynb.fs.full.dragLumFunctions import getDragLum
    from ipynb.fs.full.miscFunctions import getR2
    
    # internal
    from ipynb.fs.full.convZoneFunctions import getRadius
    from ipynb.fs.full.convZoneFunctions import findZone
    from ipynb.fs.full.convZoneFunctions import getMaxRadius
    from ipynb.fs.full.convZoneFunctions import normalizeRadius
    
    r = getRadius(p, localSim)
    tconv = getTConv(p)
    maxLum = getMaxLum(p, localSim)
    tinsp = getTInspiral(p, m2)
    dragLum = getDragLum(p, m2)
    
    # tconv is always awkwardly short
    r = r[:-2]

    maxR = getMaxRadius(r)
    n = normalizeRadius(r, maxR)
    
    coreMass = p.he_core_mass + p.c_core_mass + p.o_core_mass + p.si_core_mass + p.fe_core_mass
    coreMass = coreMass*1.989e33
    
    # cut at rshred
    r2 = getR2(m2)
    m2 = m2*1.989e33 # units
    r2 = r2*69.551e9 # units
    rshred = r2 * (2*coreMass/m2)**(1/3)
    
    i = 0
    for x in r:
        if x > rshred:
            i+=1
    
    maxLum = np.flip(maxLum)
    tinsp = np.flip(tinsp)
    
    r = r[:i]
    n = n[:i]
    tconv = tconv[:i]
    maxLum = maxLum[:i]
    tinsp = tinsp[:i]
    dragLum = dragLum[:i]

    for i in range(len(r)):
        s = findZone(tconv[i], maxLum[i], tinsp[i], dragLum[i])

        color = ''
        name = ''

        if (s == 'removeAll'):
            color = 'g+'

        elif (s == 'removeLimited'):
            color = 'b+'

        elif (s == 'removeNone'):
            color = 'r+'
            
        plt.plot(n[i], ypos, color)

In [5]:
# keep m1 constant and compare multiple m2 values

def getConvZones(p, localSim, m2):
    """Get convective zones given profile and secondary mass.
    
    Keyword arguments:
    p -- the m1 MESA profile being used
    localSim -- boolean; local or web simulation
    m2 -- mass of the secondary star
    """
    import mesa_reader as mr
    import matplotlib.pylab as plt
    import numpy as np
    import os
    from math import log
    from scipy.integrate import cumtrapz
    import math

    from ipynb.fs.full.profileFunctions import getMaxRadiusProfile
    from ipynb.fs.full.tConvFunctions import getTConv
    from ipynb.fs.full.maxLumFunctions import getMaxLum
    from ipynb.fs.full.functions import getTInspiral 
    from ipynb.fs.full.dragLumFunctions import getDragLum
    from ipynb.fs.full.miscFunctions import getR2
    
    # internal
    from ipynb.fs.full.convZoneFunctions import getRadius
    from ipynb.fs.full.convZoneFunctions import findZone
    from ipynb.fs.full.convZoneFunctions import getMaxRadius
    
    r = getRadius(p, localSim)
    tconv = getTConv(p)
    maxLum = getMaxLum(p, localSim)
    tinsp = getTInspiral(p, m2)
    dragLum = getDragLum(p, m2)
    
    # tconv is always awkwardly short
    r = r[:-2]

    maxR = getMaxRadius(r)
    
    coreMass = p.he_core_mass + p.c_core_mass + p.o_core_mass + p.si_core_mass + p.fe_core_mass
    coreMass = coreMass*1.989e33
    
    # cut at rshred
    r2 = getR2(m2)
    m2 = m2*1.989e33 # units
    r2 = r2*69.551e9 # units
    rshred = r2 * (2*coreMass/m2)**(1/3)
    
    i = 0
    for x in r:
        if x > rshred:
            i+=1
    
    maxLum = np.flip(maxLum)
    tinsp = np.flip(tinsp)
    
    r = r[:i]
    tconv = tconv[:i]
    maxLum = maxLum[:i]
    tinsp = tinsp[:i]
    dragLum = dragLum[:i]
    
    zones = []
    rFinal = []

    for i in range(len(r)):
        s = findZone(tconv[i], maxLum[i], tinsp[i], dragLum[i])
        zones.append(s)
        rFinal.append(r[i])
    
    return zones, rFinal